In [3]:
import pandas as pd
from tqdm import tqdm
from time import sleep
import matplotlib.pyplot as plt
from collections import Counter
import re
import seaborn as sns
import numpy as np

In [4]:
doctors = pd.read_excel("../data/doctors.xlsx", sheet_name='main')
nurses = pd.read_excel("../data/nurses.xlsx", sheet_name='main')

In [32]:
print("NaN values in 'doctors':", doctors.isna().sum())
print("NaN values in 'nurses':", nurses.isna().sum())

NaN values in 'doctors': year       0
region     0
private    0
total      0
public     0
dtype: int64
NaN values in 'nurses': year       0
region     0
private    0
public     0
total      0
dtype: int64


In [ ]:
# nurses.rename(columns={"НЕГОСУДАРСТВЕННАЯ ФОРМА СОБСТВЕННОСТИ": "private", "ВСЕГО (по всем формам собственности)": "total", "Государственная собственность": "public", "ВСЕГО": "total"}, inplace=True)
# doctors.rename(columns={"НЕГОСУДАРСТВЕННАЯ ФОРМА СОБСТВЕННОСТИ": "private", "ВСЕГО (по всем формам собственности)": "total", "Государственная собственность": "public", "ВСЕГО": "total"}, inplace=True)

In [33]:
doctors.head()

,year,region,private,total,public
0,2016,Российская Федерация,4760,43249,38027
1,2016,Центральный федеральный округ,1492,11404,9786
2,2016,Белгородская область,31,480,446
3,2016,Брянская область,30,324,294
4,2016,Владимирская область,38,348,307


In [ ]:
# Filter out aggregate regions
filtered = doctors[~doctors['region'].str.contains('Российская Федерация|федеральный округ')]

# Get total doctors for each region in 2016 and 2023
total_2016 = filtered[filtered['year'] == 2016].set_index('region')['total']
total_2023 = filtered[filtered['year'] == 2023].set_index('region')['total']

# Only consider regions present in both years
common_regions = total_2016.index.intersection(total_2023.index)

# Count regions where total in 2023 < total in 2016
shrunk_count = (total_2023[common_regions] < total_2016[common_regions]).sum()
print("Number of regions where total doctors shrank in 2023 vs 2016:", shrunk_count)

Number of regions where total doctors shrank in 2023 vs 2016: 66


In [5]:
# Exclude aggregate regions
filtered = doctors[~doctors['region'].str.contains('Российская Федерация|федеральный округ')]

# Pivot to have years as columns and regions as rows
pivot = filtered.pivot(index='region', columns='year', values='total')

def max_consecutive_decline(row):
    declines = (row.diff() < 0).astype(int)
    max_streak = streak = 0
    for val in declines[1:]:
        if val:
            streak += 1
            max_streak = max(max_streak, streak)
        else:
            streak = 0
    return max_streak

consecutive_decline = pivot.apply(max_consecutive_decline, axis=1)
count_3plus = (consecutive_decline >= 3).sum()
print("Number of regions with three or more years of consecutive decline:", count_3plus)

Number of regions with three or more years of consecutive decline: 42


In [35]:
# Exclude aggregate regions
filtered = doctors[~doctors['region'].str.contains('Российская Федерация|федеральный округ')]

# Pivot to have years as columns and regions as rows
pivot = filtered.pivot(index='region', columns='year', values='total')

# For each year (except the first), count regions where total decreased vs previous year
loss_counts = {}
years = sorted(pivot.columns)
for i in range(1, len(years)):
    prev, curr = years[i-1], years[i]
    loss_counts[curr] = (pivot[curr] < pivot[prev]).sum()

# Create a table
loss_table = pd.DataFrame(list(loss_counts.items()), columns=['year', 'regions_lost_doctors'])
print("Regions where total doctors decreased compared to previous year:")
print(loss_table)


Regions where total doctors decreased compared to previous year:
   year  regions_lost_doctors
0  2017                    26
1  2018                    55
2  2019                    56
3  2020                    50
4  2021                    50
5  2022                    59
6  2023                    38


In [1]:
pivot.head()

NameError: name 'pivot' is not defined

In [9]:
for year in sorted(doctors['year'].unique()):
    df_year = doctors[(doctors['year'] == year) & (~doctors['region'].str.contains('Российская Федерация')) & (~doctors['region'].str.contains('федеральный округ'))]
    min_row = df_year.loc[df_year['total'].idxmin()]
    max_row = df_year.loc[df_year['total'].idxmax()]
    print(f"{year}:")
    print(f"Lowest: {min_row['region']} - {min_row['total']}")
    print(f"Highest: {max_row['region']} - {max_row['total']}")

2016:
Lowest:             Ненецкий автономный округ (Архангельская область) - 11
Highest:         Город Москва столица Российской Федерации город федерального значения - 3994
2017:
Lowest:             Ненецкий автономный округ (Архангельская область) - 12
Highest:         Город Москва столица Российской Федерации город федерального значения - 4027
2018:
Lowest:             Ненецкий автономный округ (Архангельская область) - 14
Highest:         Город Москва столица Российской Федерации город федерального значения - 4184
2019:
Lowest:             Ненецкий автономный округ (Архангельская область) - 15
Highest:         Город Москва столица Российской Федерации город федерального значения - 4327
2020:
Lowest:             Ненецкий автономный округ (Архангельская область) - 12
Highest:         Город Москва столица Российской Федерации город федерального значения - 4655
2021:
Lowest:             Ненецкий автономный округ (Архангельская область) - 15
Highest:         Город Москва столица Россий

In [11]:
doctors.columns

Index(['year', 'region', 'private', 'total', 'public'], dtype='object')

In [15]:
# Calculate the percentage change in total doctors for each region between the earliest and latest year
region_changes = doctors[~doctors['region'].str.contains('Российская Федерация|федеральный округ')].copy()
first_year = region_changes['year'].min()
last_year = region_changes['year'].max()

first = region_changes[region_changes['year'] == first_year].set_index('region')['total']
last = region_changes[region_changes['year'] == last_year].set_index('region')['total']

# Only keep regions present in both years
common_regions = first.index.intersection(last.index)
pct_change = (last[common_regions] - first[common_regions]).sort_values()

print("Regions with largest decrease:")
print(pct_change.head(10))

print("\nRegions with largest increase:")
print(pct_change.tail(10))

Regions with largest decrease:
region
Саратовская область                                              -130
Пермский край                                                    -107
Оренбургская область                                             -104
Самарская область                                                 -95
Нижегородская область                                             -82
Кемеровская область - Кузбасс                                     -80
Алтайский край                                                    -76
Омская область                                                    -75
Хабаровский край                                                  -71
    Архангельская область (кроме Ненецкого автономного округа)    -69
Name: total, dtype: int64

Regions with largest increase:
region
Республика Ингушетия                                                       32
    Ямало-Ненецкий автономный округ (Тюменская область)                    37
Тамбовская область                       

In [20]:
doctors['region'].unique()

array(['Российская Федерация', '    Центральный федеральный округ',
       '        Белгородская область', '        Брянская область',
       '        Владимирская область', '        Воронежская область',
       '        Ивановская область', '        Калужская область',
       '        Костромская область', '        Курская область',
       '        Липецкая область', '        Московская область',
       '        Орловская область', '        Рязанская область',
       '        Смоленская область', '        Тамбовская область',
       '        Тверская область', '        Тульская область',
       '        Ярославская область',
       '        Город Москва столица Российской Федерации город федерального значения',
       '    Северо-Западный федеральный округ',
       '        Республика Карелия', '        Республика Коми',
       '        Архангельская область',
       '            Ненецкий автономный округ (Архангельская область)',
       '            Архангельская область (кроме Ненец

In [ ]:
# Exclude 'Тюменская область' but keep the specified subregions
filtered_regions_tyumen = [
    r for r in filtered_regions
    if 'тюменская область' not in r.lower().strip() or
       'ханты-мансийский автономный округ' in r.lower() or
       'ямало-ненецкий автономный округ' in r.lower() or
       'тюменская область (кроме ханты-мансийского автономного округа-югры и ямало-ненецкого автономного округа)' in r.lower()
]

In [ ]:
# Exclude specified regions and those mentioning 'российская федерация' or 'федеральный округ'
filtered_regions = [
    r for r in common_regions
    if 'российская федерация' not in r.lower()
    and 'федеральный округ' not in r.lower()
    and all(ex.strip() != r.strip() for ex in exclude)
]

gained = (last[filtered_regions] > first[filtered_regions]).sum()
lost = (last[filtered_regions] < first[filtered_regions]).sum()
unchanged = (last[filtered_regions] == first[filtered_regions]).sum()

print(f"Regions that gained doctors: {gained}")
print(f"Regions that lost doctors: {lost}")
print(f"Regions with no change: {unchanged}")


Regions that gained doctors: 15
Regions that lost doctors: 64
Regions with no change: 1


In [25]:
filtered_regions

['        Белгородская область',
 '        Брянская область',
 '        Владимирская область',
 '        Воронежская область',
 '        Ивановская область',
 '        Калужская область',
 '        Костромская область',
 '        Курская область',
 '        Липецкая область',
 '        Московская область',
 '        Орловская область',
 '        Рязанская область',
 '        Смоленская область',
 '        Тамбовская область',
 '        Тверская область',
 '        Тульская область',
 '        Ярославская область',
 '        Город Москва столица Российской Федерации город федерального значения',
 '        Республика Карелия',
 '        Республика Коми',
 '        Архангельская область',
 '            Ненецкий автономный округ (Архангельская область)',
 '            Архангельская область (кроме Ненецкого автономного округа)',
 '        Вологодская область',
 '        Калининградская область',
 '        Ленинградская область',
 '        Мурманская область',
 '        Новгородская область'

In [18]:
doctors.loc[(doctors['region'] == 'Чукотский автономный округ') & (doctors['year'] == 2023), 'total']

Series([], Name: total, dtype: int64)

### Nurses

In [35]:
nurses.dtypes

year         int64
region      object
private    float64
public       int64
total        int64
dtype: object

In [36]:
for year in sorted(nurses['year'].unique()):
    df_year = nurses[(nurses['year'] == year) & (~nurses['region'].str.contains('Российская Федерация|федеральный округ'))]
    min_row = df_year.loc[df_year['total'].idxmin()]
    max_row = df_year.loc[df_year['total'].idxmax()]
    print(f"{year}:")
    print(f"Lowest: {min_row['region']} - {min_row['total']}")
    print(f"Highest: {max_row['region']} - {max_row['total']}")

2016:
Lowest:             Ненецкий автономный округ (Архангельская область) - 20
Highest:         Город Москва столица Российской Федерации город федерального значения - 2829
2017:
Lowest:             Ненецкий автономный округ (Архангельская область) - 18
Highest:         Город Москва столица Российской Федерации город федерального значения - 2928
2018:
Lowest:             Ненецкий автономный округ (Архангельская область) - 19
Highest:         Город Москва столица Российской Федерации город федерального значения - 2891
2019:
Lowest:             Ненецкий автономный округ (Архангельская область) - 18
Highest:         Город Москва столица Российской Федерации город федерального значения - 3000
2020:
Lowest:             Ненецкий автономный округ (Архангельская область) - 17
Highest:         Город Москва столица Российской Федерации город федерального значения - 2999
2021:
Lowest:             Ненецкий автономный округ (Архангельская область) - 22
Highest:         Город Москва столица Россий

In [8]:
# Calculate the percentage change in total nurses for each region between the earliest and latest year
region_changes = nurses[~nurses['region'].str.contains('Российская Федерация|федеральный округ')].copy()
first_year = region_changes['year'].min()
last_year = region_changes['year'].max()

first = region_changes[region_changes['year'] == first_year].set_index('region')['total']
last = region_changes[region_changes['year'] == last_year].set_index('region')['total']

# Only keep regions present in both years
common_regions = first.index.intersection(last.index)
pct_change = ((last[common_regions] - first[common_regions]) / first[common_regions] * 100).sort_values()

print("Regions with largest percentage decrease:")
print(pct_change.head(10))

print("\nRegions with largest percentage increase:")
print(pct_change.tail(10))

Regions with largest percentage decrease:
region
Город федерального значения Севастополь                          -63.009404
Новгородская область                                             -56.146179
Еврейская автономная область                                     -42.222222
Псковская область                                                -41.409692
Удмуртская Республика                                            -39.335477
Республика Калмыкия                                              -38.125000
Кемеровская область - Кузбасс                                    -37.430168
    Архангельская область (кроме Ненецкого автономного округа)   -37.401575
Архангельская область                                            -36.553030
Тверская область                                                 -36.178862
Name: total, dtype: float64

Regions with largest percentage increase:
region
Калининградская область                                                                                         -

In [9]:
# Calculate the change in total nurses for each region between the earliest and latest year
region_changes = nurses[~nurses['region'].str.contains('Российская Федерация|федеральный округ')].copy()
first_year = region_changes['year'].min()
last_year = region_changes['year'].max()

first = region_changes[region_changes['year'] == first_year].set_index('region')['total']
last = region_changes[region_changes['year'] == last_year].set_index('region')['total']

# Only keep regions present in both years
common_regions = first.index.intersection(last.index)
pct_change = (last[common_regions] - first[common_regions]).sort_values()

print("Regions with largest percentage decrease:")
print(pct_change.head(10))

print("\nRegions with largest percentage increase:")
print(pct_change.tail(10))

Regions with largest percentage decrease:
region
Республика Башкортостан            -508
Краснодарский край                 -484
Кемеровская область - Кузбасс      -402
Удмуртская Республика              -367
Республика Татарстан (Татарстан)   -359
Нижегородская область              -355
Волгоградская область              -340
Алтайский край                     -316
Ставропольский край                -295
Свердловская область               -278
Name: total, dtype: int64

Regions with largest percentage increase:
region
Калининградская область                                                                                         -14
Магаданская область                                                                                             -12
    Ямало-Ненецкий автономный округ (Тюменская область)                                                          -5
    Ненецкий автономный округ (Архангельская область)                                                            -3
Чукотский а